In [47]:
import pandas as pd
import numpy as np
import sys,csv,glob

In [13]:
filteredGeneList = pd.read_csv('../../Database/filteredGenesDetails_human_240118.txt',sep='\t')

In [14]:
acc2gene={}
gene2acc={}
for idx,row in filteredGeneList.iterrows():
    acc2gene[row['AccNum']] = row['GeneName']
    gene2acc[row['GeneName']] = row['AccNum']

In [15]:
def get_fa(filename,onlyKeys = 'All'):
    keyD = dict()
    if onlyKeys!='All':
        for k in onlyKeys: keyD[k] = None
        
    def addEntry(current,d):
        entry = current.split('\n')
        if entry[0]!='' and (onlyKeys=='All' or keyD.has_key(entry[0])):
            d[entry[0]] = ''.join(entry[1:])

    d = dict()
    bf = open(filename)
    current = ['\n']
    for line in bf:
        if line[0]=='>':
            addEntry(''.join(current),d)
            if len(line)>1: current = [line[1:]]
            else: current = ['\n']
        else: current.append(line)
    addEntry(''.join(current),d)
    bf.close()
    return d

In [20]:
splicedGenes = get_fa('../../Database/splicedGenes_240118.txt')
splicedORFs = get_fa('../../Database/splicedORFs_240118.txt')

In [21]:
ks = list(splicedGenes.keys())
for key in ks:
    splicedGenes[key.split('_chr')[0]] = splicedGenes[key]
    splicedGenes.pop(key)

In [22]:
fpUTR={}
tpUTR = {}
ORFs = {}
for idx,row in filteredGeneList.iterrows():
    accNum = row['AccNum']
    fpLen = row['fpUTR_length']
    tpLen = row['tpUTR_length']
    fpUTR[accNum] = splicedGenes[accNum][0:fpLen]
    tpUTR[accNum] = splicedGenes[accNum][-tpLen:]
    ORFs[accNum] = splicedORFs[accNum]

In [2]:
with open('fpUTR_rnafold.out') as f:
    lines = [line.rstrip() for line in f]

In [11]:
print(len(lines)/6.0)
lines[:6]  ##every 6 is one set


18316.0


['>UBE2Q1',
 'CGGCGGCGGCGGCGGCGGCGGCGGCGGCGGAGCCCGGAGCGCAGGCCGGAGGCUCCCGGCCCGCCGGCCCCGGAGCGGAGCGGAGCGGAGG',
 '..((.((.((....)).)).))..((((((.(((.(((((...........))))).)))))))))((.(((........))).))..... (-47.40)',
 '..((.{(.{(..,.}|.}}{|}{{((((((.(((.(((((...........))))).))))))))}}))|||..}|...,)}..}),.... [-48.98]',
 '......................((((((((.(((.(((((...........))))).))))))))).))...................... {-31.50 d=13.56}',
 ' frequency of mfe structure in ensemble 0.077272; ensemble diversity 20.82']

In [26]:
## verify if it is always 6 lines per entry
for i in range(18316):
    if lines[i*6][0]!= '>':
        print(i)

In [63]:
acc2energy={}
for i in range(len(lines)//6):
    geneName = lines[i*6][1:]
    acc = gene2acc[geneName]
    theline = lines[i*6+3]
    linesep = theline.split('[')
    val = float(linesep[1][:-1])
    acc2energy[acc]=val

In [64]:
print(len(acc2energy))
print(len(filteredGeneList))

18316
19210


In [65]:
## norm by length
if True:
    for key in acc2energy.keys():
        fpLen = len(fpUTR[key])
        acc2energy[key] = acc2energy[key] / fpLen

In [66]:
mean_ = np.array(list(acc2energy.values())).mean()
print('mean:',mean_)
## fill with mean
for idx,row in filteredGeneList.iterrows():
    accNum = row['AccNum']
    if accNum not in acc2energy:
        acc2energy[accNum]=mean_

mean: -0.3766069317021581


In [67]:
print(len(acc2energy))

19210


In [68]:
## to csv
with open('fpUTR_minE_normbyLen_fillmean.csv', 'w') as f:  # Just use 'w' mode in 3.x
    f.write('AccNum,Energy\n')
    for key in acc2energy.keys():
        f.write("%s,%s\n"%(key,acc2energy[key]))

In [69]:
with open('utr_orf100.out') as f:
    lines = [line.rstrip() for line in f]

In [78]:
acc2energy={}
for i in range(len(lines)//6):
    geneName = lines[i*6][1:]
    acc = gene2acc[geneName]
    theline = lines[i*6+3]
    linesep = theline.split('[')
    val = float(linesep[1][:-1])
    acc2energy[acc]=val
print(len(acc2energy))

19210


In [79]:
## norm by length
if True:
    for key in acc2energy.keys():
        fpLen = len(fpUTR[key])
        acc2energy[key] = acc2energy[key] / (fpLen+100)

In [80]:
## to csv
with open('utr_and_orf_minE_normbyLen.csv', 'w') as f:  # Just use 'w' mode in 3.x
    f.write('AccNum,Energy\n')
    for key in acc2energy.keys():
        f.write("%s,%s\n"%(key,acc2energy[key]))

In [88]:
L2 = pd.read_csv('../../../../../Downloads/list1.csv')

In [89]:
L2.head()

,GeneSymbol,Entrez_Gene_ID,Gene Name,Avg_Zpl_RateAll,SE_Avg_Zpl_RateAll,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,Unnamed: 10
0,SCARB2,950.0,"scavenger receptor class B, member 2",-4.114580,0.595995,NaN,NaN,NaN,NaN,NaN,NaN
1,COPB2,9276.0,"coatomer protein complex, subunit beta 2 (beta...",-4.045615,0.638425,NaN,NaN,NaN,NaN,NaN,NaN
2,KRTAP5-2,440021.0,keratin associated protein 5-2,-3.839167,0.723455,NaN,NaN,NaN,NaN,NaN,NaN
3,BLOC1S1,2647.0,"biogenesis of lysosomal organelles complex-1, ...",-3.701800,0.660136,NaN,NaN,NaN,NaN,NaN,NaN
4,TAAR6,319100.0,trace amine associated receptor 6,-3.695076,0.638628,NaN,NaN,NaN,NaN,NaN,NaN


In [93]:
L1 = pd.read_csv('../../../../../Downloads/Gene2.txt',header=None)

In [94]:
L1.head()

,0
0,F2RL2
1,ITPR3
2,ZFHX3
3,UCP2
4,ARHGAP29


In [99]:
set(L1[0]).intersection(set(L2['GeneSymbol']))

{'ABL1',
 'BTAF1',
 'HACE1',
 'IGSF3',
 'KIFC2',
 'KSR1',
 'MINK1',
 'PHF14',
 'SSH1',
 'TFDP1',
 'TIAM2',
 'TNK2',
 'TRIO',
 'USP36',
 'WDR24',
 'XPO6'}